# Adding a new game to the framework - FirstLast

- The players should engage in a turn-based conversation about a predefined topic. 
- Player A starts with an utterance whose first and last token must start with a `predefined letter`, say d. 
- Player B must then reply with an utterance whose first and last token must be the `next one in the alphabet` (here, an e). <br> <br>
And so on, for n turns **(where each turn is comprised by an utterance from A and an utterance from B)**.<br> <br>
- If an utterance does not conform to these rules (i.e. it is incorrect), the players lose the game. <br>
- `move rule:` If an utterance does not start with 'I SAY: ' (i.e., it is invalid), the game is immediately aborted. <br>
- If all utterances up to turn n are valid and correct, the game is successful.

For instance, if the topic is **birds**, the initial letter is `h` and the number of turns is 2, this would be a successful game:

- Hi! I love birds, but it's hard to identify them. I need help. `(h: hi / help)`
- I know what you mean. I can try to help, please describe it. `(i: I/ it)`
- Just a moment... Ok, it's blue but looks like an Eurasian jay. `(j: just / jay)`
- Kick in more details, otherwise I don't know. `(k: kick / know)`

Each turn, we need to check two aspects: 
- **MOVE_RULE**: Does the utterance start with 'I SAY'?
- **GAME_RULE**: Do the first/last tokens start with `predefined_letter`?

# Adding a new game:
We will need at least the following components:
- **Game resources:** all data, prompt templates and text files that are necessary to create instances of a game and to group these instances into experiments.
- **Instances:** JSON containing the configuration of each instance, grouped into experiments. 
    - This must be done by a script named `instancegenerator.py`, with a class that inherits from `GameInstanceGenerator`.
- **Game Master:** Controls and enforces MOVE/GAME Rules, inheriting from GameMaster. 
    - This must be implemented in a file `master.py`.
- **Players:** Defines the programatic behaviour and any other attributes of a player, inheriting from `Player`.
    - This can be implemented in a file named `players.py`.
- **Game Benchmark:** a class that realises the game, inheriting from `GameBenchmark`. 
    - This can also live in the file `master.py`.

To define an **Episode**, we have to instantiate the initial promopts and define 3 more parameters:
- Topic
- Letter
- Number of Turns

# Defining prompts with game rules
- In the prompt template we can define variables that can be replaced later.
- Prompts have to be adjusted for Player A and B, based on their roles.

In [ ]:
path = Path('resources') / 'initial_prompts'

with open(path / 'initial_prompt_a.template', 'w') as file:
    file.write(
        "Let's play a game. You must have a conversation about $topic with your partner. Your first turn must start and end with words that begin with the letter $letter. The reply of your partner must be similar, with the letter that comes after $letter in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for $nturns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."
    )

with open(path / 'initial_prompt_b.template', 'w') as file:
    file.write(
        "Let's play a game. You must have a conversation about $topic with your partner. Their first turn must start and end with words that begin with the letter $letter. Your reply must be similar, with the letter that comes after $letter in the alphabet. Then it's their turn again with the next letter, and so on. You'll do it for $nturns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."
    )

# Save topics to topics.txt
topics = ['dogs', 'cats', 'birds', 'trees']
with open(path / 'topics.txt', 'w') as file:
    for topic in topics:
        file.write(topic + '\n') 


# Creating game instances
- instancegenerator.py will create instances.json
- Create Class that inherits from GameInstanceGenerator and define `_on_generate`
- In main: Instantiate Class and call `.generate()`method

- `_on_generate:` Define experiments and instances, based on what dimensions we want to evaluate later.
- `Experiment` Set of instances with the same topic
    - `Instance:`
        - Initial Letter
        - Initial Prompt
        - Number of turns

The instances.json file should contain everything that the game master needs to set up the configuration of a game play!
#### Example:

In [ ]:
{
    "experiments": [
        {
            "name": "NAME_1",
            "game_instances": [
                {
                    "game_id": 0,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
                {
                    "game_id": 1,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
            ]
        },
        {
            "name": "NAME_2",
            "game_instances": [
                {
                    "game_id": 0,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
                {
                    "game_id": 1,
                    "first_letter": "LETTER",
                    "n_turns": "N",
                    "prompt_player_a": "PROMPT_A",
                    "prompt_player_b": "PROMPT_B",
                },
            ]
        },
    ]
}

In [ ]:
# save the contents of this cell as games/firstlast/instancegenerator.py
import random
import string

from clemgame.clemgame import GameInstanceGenerator

# set the name of the game in the script, as you named the directory
# this name will be used everywhere, including in the table of results
GAME_NAME = 'firstlast'
# we will create 10 instances for each experiment; vary this as you wish
N_INSTANCES = 10
# if the generation involves randomness, remember to set a random seed
SEED = 123

class FirstLastGameInstanceGenerator(GameInstanceGenerator):
    def __init__(self):
        # GameInstanceGenerator
        super().__init__(GAME_NAME)
    
    # define on_generate, a mandatory method
    def on_generate(self):
        # get the list of topics, which will be our experiments
        topics = self.load_file('resources/topics.txt').strip('\n').split('\n')
        # get the prompts for player a and player b
        # we'll keep the prompts fixed in all instances, replacing only the
        # necessary slots (but you can do it differently)
        prompt_a = self.load_template('resources/initial_prompts/initial_prompt_a')
        prompt_b = self.load_template('resources/initial_prompts/initial_prompt_b')

        # building the file, one experiment at a time
        for topic in topics:
            # create an experiment (for us, named after a topic)
            experiment = self.add_experiment(topic)
            # build N_INSTANCES instances for each experiment
            for game_id in range(N_INSTANCES):
                # set the parameters
                # here we do it randomly, but that can also be read from a file
                # one of the first 5 letters in the alphabet
                letter = random.choice(string.ascii_lowercase[:5])
                # up to 8 turns, so that we don't run out of letters
                n_turns = random.randint(3, 8)
                # create a game instance, using a game_id counter/index
                instance = self.add_game_instance(experiment, game_id)
                # populate the game instance with its parameters
                instance['first_letter'] = letter
                instance['n_turns'] = n_turns
                instance['prompt_player_a'] = self.create_prompt(
                    topic, prompt_a, letter, n_turns)
                instance['prompt_player_b'] = self.create_prompt(
                    topic, prompt_b, letter, n_turns)
    
    # an additional method, specific for our example
    def create_prompt(self,
                      topic: str,
                      prompt: str,
                      letter: str,
                      n_turns: int) -> str:
        """Replace a prompt template with slot values."""
        text = string.Template(prompt).substitute(topic=topic, letter=letter,
                                                  nturns=n_turns)
        return text


if __name__ == '__main__':
    random.seed(SEED)
    # always call this, which will actually generate and save the JSON file
    FirstLastGameInstanceGenerator().generate()

#### Results

In [ ]:
{
    "experiments": [
        {
            "name": "dogs",
            "game_instances": [
                {
                    "game_id": 0,
                    "first_letter": "a",
                    "n_turns": 5,
                    "prompt_player_a": "Test Prompt A",
                    "prompt_player_b": "Test Prompt B"
                },
                {
                    "game_id": 1,
                    "first_letter": "a",
                    "n_turns": 6,
                    "prompt_player_a": "Test Prompt A",
                    "prompt_player_b": "Test Prompt B"
                },
                {
                    "game_id": 2,
                    "first_letter": "c",
                    "n_turns": 3,
                    "prompt_player_a": "Test Prompt A",
                    "prompt_player_b": "Test Prompt B"
                },
                {
                    "game_id": 3,
                    "first_letter": "a",
                    "n_turns": 6,
                    "prompt_player_a": "Test Prompt A",
                    "prompt_player_b": "Test Prompt B"
                }
            ]
        }
    ]
}

# Creating the Game